This example shows how to link a point to the NHDPlusV2.1 and NHD High Resolution.

For this example we use a point near the Red Cedar River in East Lansing, MI.

##### First set required parameters.
The same input is the same for NHDPlusV2.1 and NHD High Resolution methods.

Required parameters include:
    *input_identifier: str, user supplied identifier
    *input_lat: float, latitude of the point to be hydrolinked
    *input_lon: float, longitude of the point to be hydrolinked
    
Other optional parameters include:
    *input_crs: int, coordinate reference system, default is 4269 (NAD83)
    *stream_name: name of the stream a point is intended to be linked
    *buffer_m = search distance in meters, default is 1000, max is 2000

In [7]:
#identifier -> number or string
input_identifier = 1
#latitude
input_lat = 42.7284
#longitude
input_lon = -84.5026
#coordinate system
input_crs = 4269  
#stream name
stream_name = 'The Red Cedar River'
#buffer
buffer_m = 2000

#### We will first hydrolink to the NHDPlusV2.1 (Medium Resolution)

First we import the module, initiate a Python object called hydrolink (which will store relevant information) and attempt to set the coordinate system to NAD83 (crs = 4269).

Note: It is recommended to use crs 4269, although it was attempted to accomidate other common coordinate systems.  The methods will let you know if your coordinate system is causing issues.

In [8]:
#Import module from local folder hydrolink
from hydrolink import nhd_mr

#initiate a Python object
hydrolink = nhd_mr.MedResPoint(input_identifier, input_lat, input_lon, stream_name=stream_name, input_crs=input_crs, buffer_m=buffer_m)

#We then try setting the coordinate system to NAD83
hydrolink.crs_to_4269()

The next cell runs three steps:

* First it builds a query to return reachcodes within buffer of point location. Medium resolution uses EPA rest services to return reaches of interest: https://inlandwaters.geoplatform.gov/arcgis/rest/services/NHDPlus/NHDPlus/MapServer  
Within this service in-network flowlines (network_flow) of off-network flowlines (nonnetwork flow) can be specified with the variable service.  

* Next we execute the query.  User has the option to set the parameter n to specify max number of reachcodes to return.  During this step we measure distances from the user defined point to each reach and from the user defined point to the nearest node representing a confluence.  

* Next we calculate a score of how well the stream name matches (user supplied vs. each reach GNIS_NAME). Results are output in name_check variables.  name_check is a value between 0 and 1 where 0 represents no match and 1 represents and exact match. A fuzzy match algorithm is used (difflib) where names are not an exact match and therefor this value is a continuous number.

In [10]:
#builds query against EPA web services
hydrolink.build_reach_query(service='network_flow')

#executes query and measures distances from point to each line and their nodes
hydrolink.find_closest_reaches(n=6)

#stream name match for each of the reaches
hydrolink.stream_match()

#show result, there were only 3 reaches within 2000 meters of this point
hydrolink.reach_df

,GNIS_NAME,LENGTHKM,PERMANENT_IDENTIFIER,REACHCODE,COMID,closest_node_m,snap_m,snap_xy,closest,name_check,name_check_txt
0,Red Cedar River,8.421667,12242482,04050004000126,12242482,1987.423043,44.332661,POINT (-84.50243779700608 42.72877868601643),1,0.882353,most likely match of stream names based on fuz...
1,None,1.983866,12241744,04050004000960,12241744,1987.423043,1837.753163,POINT (-84.50737777864552 42.71232846672766),2,0.000000,stream name and or gnis name not provided
2,Red Cedar River,0.790011,12241734,04050004000125,12241734,1987.423043,1987.423043,POINT (-84.52110377914246 42.71680106720647),3,0.882353,most likely match of stream names based on fuz...


We can then select the best reach from our options using the following criteria.
 
 Of returned reaches finds most likely reach based on available information
        * First checks to see if any reaches have an exact match of names
            * If number of reaches with exact name match equals 1 that is the reach to recommend
            * If number of reaches with exact name match > 1 take the one that is closest to the point
               * If more than one reach has exact name match and are equally close to the point grab the first but note that multiple reaches so that we can recommend taking a closer look
        * If no reaches with exact name match then check for fuzzy matches over 0.75 cutoff
        * If number of reaches with fuzzy name match equals 1 that is the reach to recommend
        * If number of reaches with fuzzy name match > 1 take the one that is closest to the point
            * If more than one reach has fuzzy name match and are equally close to the point grab the first one but note that multiple reaches so that we can recommend taking a closer look
        * If fuzzy match < 0.75 just take closest reach.

In [12]:
hydrolink.select_best_reach()

#show results
hydrolink.best_reach

{'init_id': 1,
 'init_stream': 'The Red Cedar River',
 'stream_clean_ref': 'the red cedar river',
 'GNIS_NAME': 'Red Cedar River',
 'LENGTHKM': 8.42166733289811,
 'PERMANENT_IDENTIFIER': '12242482',
 'COMID': 12242482,
 'REACHCODE': '04050004000126',
 'snap_xy': <shapely.geometry.point.Point at 0x2459b93e348>,
 'snap_m': 44.332660593521666,
 'closest_node_m': 1987.4230429025345,
 'closest': 1,
 'name_check': 0.8823529411764706,
 'mult_reach_ct': 1,
 'name_check_txt': 'most likely match of stream names based on fuzzy match',
 'mr_meas': None,
 'message': ''}

option to get address of point for the best reach

In [ ]:
hydrolink.get_hl_measure()

we can then write the data out to a csv file

In [ ]:

hydrolink.write_reach_options()

hydrolink.write_best()